In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


In [2]:
data=pd.read_csv("../merged/len3_ndb_agg_blp_DropNever.csv",index_col=0,encoding="shift-jis")
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_399.0,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,r_share,generic_share_q,generic_share_r
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,0,0,0,0,0,0,0.007258,0.002068,0.236142,0.053889
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,0,0,0,0,0,0,0.002987,0.001507,0.236142,0.053889
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,0,0,0,0,0,0,0.014653,0.004175,0.236142,0.053889
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,0,0,0,0,0,0,0.007997,0.004036,0.236142,0.053889
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,0,0,0,0,0,0,0.001557,0.002136,0.236142,0.053889


In [3]:
data=data.drop_duplicates(subset=["薬効分類","year"])
# drop never treated group
data.head(),data.shape

(      薬効分類   薬効分類名称       医薬品コード          医薬品名  薬価基準収載医薬品コード    薬価  後発品区分  \
 0    114.0  解熱鎮痛消炎剤  620007096.0    ボルタレン錠２５ｍｇ  1147002F1560  13.1    0.0   
 80   121.0    局所麻酔剤  620005991.0    ストロカイン錠５ｍｇ  1219002F1065   5.7    0.0   
 96   123.0    自律神経剤  620002526.0  チアトンカプセル１０ｍｇ  1231013M2179  15.2    0.0   
 147  131.0     眼科用剤  620207901.0   アダプチノール錠５ｍｇ  1319004F1035  45.9    0.0   
 199  132.0    耳鼻科用剤  620004822.0    プリビナ液０．０５％  1324704Q1033   4.5    0.0   
 
                総計  year  in_hospital  ...  id_399.0  id_441.0  id_449.0  \
 0    2.489539e+07  2014            1  ...         0         0         0   
 80   1.161969e+07  2014            1  ...         0         0         0   
 96   9.128303e+06  2014            1  ...         0         0         0   
 147  5.229358e+06  2014            1  ...         0         0         0   
 199  6.506744e+06  2014            1  ...         0         0         0   
 
     id_625.0 id_629.0 id_799.0   q_share   r_share generic_share_q  \
 

In [4]:
data_map=pd.read_csv("../generic/generic_usage_imp.csv")
generic_per_map={}
for i in range(data_map.shape[0]):
    generic_per_map[data_map.iloc[i,0]]=data_map.iloc[i,1]
generic_per_map[2023]=83
generic_per_map

{2007: 34.9,
 2008: 35.35,
 2009: 35.8,
 2010: 37.85,
 2011: 39.9,
 2012: 43.4,
 2013: 46.9,
 2014: 51.55,
 2015: 56.2,
 2016: 61.0,
 2017: 65.8,
 2018: 72.6,
 2019: 76.7,
 2020: 78.3,
 2021: 79.0,
 2022: 79.0,
 2023: 83}

In [5]:
# set(data["薬効分類"])
data.fillna(0,inplace=True)
data[data["generic_share_q"].isnull()]

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_399.0,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,r_share,generic_share_q,generic_share_r


In [6]:
data=data.loc[:,["薬効分類","year","generic_per",'generic_share_q', 'generic_share_r']]
final_cols=["薬効分類","year",'generic_share_q', 'generic_share_r']
final_cols

['薬効分類', 'year', 'generic_share_q', 'generic_share_r']

In [7]:
# 33 ids exist
add_df=pd.DataFrame(columns=data.columns)
for id in set(data["薬効分類"]):
    for year in range(2008,2024):
        if year not in data[data["薬効分類"]==id]["year"].values:
            # 229 の2018までと224の2017以降は0
            if id==229 and year<=2018:
                add_df=add_df.append(pd.DataFrame([[id,year,generic_per_map[year]]+[0.0]*(data.shape[1]-3)],columns=data.columns),ignore_index=True)
            elif id==224 and year>=2017:
                add_df=add_df.append(pd.DataFrame([[id,year,generic_per_map[year]]+[0.0]*(data.shape[1]-3)],columns=data.columns),ignore_index=True)
            else:
                add_df=add_df.append(pd.DataFrame([[id,year,generic_per_map[year]]+[np.nan]*(data.shape[1]-3)],columns=data.columns),ignore_index=True)
add_df.sort_values(by=["薬効分類","year"],inplace=True)
add_df.reset_index(drop=True,inplace=True)
add_df,add_df.shape

C:\Users\hakos\AppData\Local\Temp\ipykernel_6088\1021681621.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  add_df=add_df.append(pd.DataFrame([[id,year,generic_per_map[year]]+[np.nan]*(data.shape[1]-3)],columns=data.columns),ignore_index=True)
C:\Users\hakos\AppData\Local\Temp\ipykernel_6088\1021681621.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  add_df=add_df.append(pd.DataFrame([[id,year,generic_per_map[year]]+[0.0]*(data.shape[1]-3)],columns=data.columns),ignore_index=True)


(      薬効分類  year generic_per generic_share_q generic_share_r
 0    114.0  2008       35.35             NaN             NaN
 1    114.0  2009        35.8             NaN             NaN
 2    114.0  2010       37.85             NaN             NaN
 3    114.0  2011        39.9             NaN             NaN
 4    114.0  2012        43.4             NaN             NaN
 ..     ...   ...         ...             ...             ...
 195  799.0  2011        39.9             NaN             NaN
 196  799.0  2012        43.4             NaN             NaN
 197  799.0  2013        46.9             NaN             NaN
 198  799.0  2022        79.0             NaN             NaN
 199  799.0  2023          83             NaN             NaN
 
 [200 rows x 5 columns],
 (200, 5))

In [8]:
dummies=pd.get_dummies(data["薬効分類"])
data=pd.concat([data,dummies],axis=1)
data

,薬効分類,year,generic_per,generic_share_q,generic_share_r,114.0,121.0,123.0,131.0,132.0,...,264.0,265.0,332.0,339.0,399.0,441.0,449.0,625.0,629.0,799.0
0,114.0,2014,51.55,0.236142,0.053889,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,121.0,2014,51.55,0.062114,0.053451,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,123.0,2014,51.55,0.129137,0.064243,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
147,131.0,2014,51.55,0.136869,0.080471,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
199,132.0,2014,51.55,0.026413,0.058417,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25142,441.0,2021,79.00,0.342768,0.263456,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
25182,449.0,2021,79.00,0.608051,0.363450,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
25386,625.0,2021,79.00,0.502607,0.052158,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
25628,629.0,2021,79.00,0.284349,0.073842,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
dummies=pd.get_dummies(add_df["薬効分類"])
add_df=pd.concat([add_df,dummies],axis=1)
add_df

c:\Users\hakos\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,薬効分類,year,generic_per,generic_share_q,generic_share_r,114.0,121.0,123.0,131.0,132.0,...,264.0,265.0,332.0,339.0,399.0,441.0,449.0,625.0,629.0,799.0
0,114.0,2008,35.35,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,114.0,2009,35.8,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,114.0,2010,37.85,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,114.0,2011,39.9,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,114.0,2012,43.4,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,799.0,2011,39.9,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
196,799.0,2012,43.4,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
197,799.0,2013,46.9,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
198,799.0,2022,79.0,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
# data_est
# ids=data.iloc[:,4:]
# X=data[["year"]+list(ids.columns)]
# y=data["generic_share_q"]
# # # X = sm.add_constant(X)  # 定数項を追加
# # model = sm.OLS(y, X).fit()
# X


In [11]:
data=data.append(add_df,ignore_index=True)
data.sort_values(by=["薬効分類","year"],inplace=True)
data.reset_index(drop=True,inplace=True)
data

C:\Users\hakos\AppData\Local\Temp\ipykernel_6088\1296814082.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(add_df,ignore_index=True)


,薬効分類,year,generic_per,generic_share_q,generic_share_r,114.0,121.0,123.0,131.0,132.0,...,264.0,265.0,332.0,339.0,399.0,441.0,449.0,625.0,629.0,799.0
0,114.0,2008,35.35,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,114.0,2009,35.8,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,114.0,2010,37.85,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,114.0,2011,39.9,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,114.0,2012,43.4,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,799.0,2019,76.7,0.006196,0.024777,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
396,799.0,2020,78.3,0.006056,0.022847,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
397,799.0,2021,79.0,0.007953,0.028184,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
398,799.0,2022,79.0,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
for target in ["generic_share_q","generic_share_r"]:
    data_est=data.dropna(subset=[target])
    ids=data_est.iloc[:,5:]
    X=data_est[["year"]+["generic_per"]+list(ids.columns)]
    y=data_est[target]
    # X = sm.add_constant(X)  # 定数項を追加
    model = sm.Logit(y.astype(float), X.astype(float)).fit()

    # モデルの要約を表示
    print(model.summary())
    # X.shape,y.shape
    missing=data[data[target].isnull()]
    missing=missing[["year","generic_per"]+list(ids.columns)].astype(float)
    # missing["const"]=1
    predicted_values=model.predict(missing)
    # predicted_values
    missing[target]=predicted_values
    data.update(missing)


Optimization terminated successfully.
         Current function value: 0.451334
         Iterations 22
                           Logit Regression Results                           
Dep. Variable:        generic_share_q   No. Observations:                  206
Model:                          Logit   Df Residuals:                      179
Method:                           MLE   Df Model:                           26
Date:                Sat, 09 Sep 2023   Pseudo R-squ.:                 0.04357
Time:                        21:57:51   Log-Likelihood:                -92.975
converged:                       True   LL-Null:                       -97.210
Covariance Type:            nonrobust   LLR p-value:                    0.9995
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
year            0.1174      0.358      0.328      0.743      -0.585       0.819
generic_per     0.0117   

c:\Users\hakos\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [13]:
data=data[final_cols]

In [14]:
data.to_csv("../merged/len3_ndb_generic_imputed.csv",encoding="shift-jis")